In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.types import * 
from pyspark.sql import SparkSession, DataFrame as SparkDataFrame
import pyspark.sql.functions as F
from pyspark.sql.functions import col,isnan, when, count, coalesce
from pyspark import SparkContext
from pyspark.sql import SQLContext
import json
from functools import reduce
import sys
from cassandra.cluster import Cluster
import os
# from mock.tasks import adiciona_carro
cluster = Cluster(['cassandra'])
session = cluster.connect()

ss = SparkSession.builder.appName("test").getOrCreate()
sql = SQLContext(ss)

/usr/local/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [32]:
session.execute("USE simulacao")

In [33]:
session.execute("select count(*) from simulacao")[0]

/tmp/ipykernel_344/795349805.py:1: DeprecationWarning: ResultSet indexing support will be removed in 4.0. Consider using ResultSet.one() to get a single row.
  session.execute("select count(*) from simulacao")[0]


Row(count=3134)

In [34]:
result = session.execute("SELECT * FROM simulacao WHERE tempo_da_simulacao = 142 ALLOW FILTERING;")

In [35]:
result[0]

/tmp/ipykernel_344/223460175.py:1: DeprecationWarning: ResultSet indexing support will be removed in 4.0. Consider using ResultSet.one() to get a single row.
  result[0]


Row(rodovia='BR-116', tempo_da_simulacao=142, placa='ARG5B67', pos_x=655.0, pos_y=168.99999999999872)

In [36]:
r = list(result)

In [37]:
r

[Row(rodovia='BR-116', tempo_da_simulacao=142, placa='ARG5B67', pos_x=655.0, pos_y=168.99999999999872),
 Row(rodovia='BR-116', tempo_da_simulacao=142, placa='CHI6S78', pos_x=745.0, pos_y=820.0),
 Row(rodovia='BR-116', tempo_da_simulacao=142, placa='ECU6I78', pos_x=565.0, pos_y=90.75),
 Row(rodovia='BR-116', tempo_da_simulacao=142, placa='GUY1X23', pos_x=385.0, pos_y=310.25000000000017),
 Row(rodovia='BR-116', tempo_da_simulacao=142, placa='PAR7P89', pos_x=475.0, pos_y=152.25000000000003),
 Row(rodovia='BR-116', tempo_da_simulacao=142, placa='SUR8W90', pos_x=745.0, pos_y=749.15),
 Row(rodovia='BR-116', tempo_da_simulacao=142, placa='URU0A12', pos_x=835.0, pos_y=650.5),
 Row(rodovia='BR-116', tempo_da_simulacao=142, placa='URU4O56', pos_x=565.0, pos_y=869.2500000000007)]

In [38]:
df = ss.createDataFrame(r)

df.show()

+-------+------------------+-------+-----+------------------+
|rodovia|tempo_da_simulacao|  placa|pos_x|             pos_y|
+-------+------------------+-------+-----+------------------+
| BR-116|               142|ARG5B67|655.0|168.99999999999872|
| BR-116|               142|CHI6S78|745.0|             820.0|
| BR-116|               142|ECU6I78|565.0|             90.75|
| BR-116|               142|GUY1X23|385.0|310.25000000000017|
| BR-116|               142|PAR7P89|475.0|152.25000000000003|
| BR-116|               142|SUR8W90|745.0|            749.15|
| BR-116|               142|URU0A12|835.0|             650.5|
| BR-116|               142|URU4O56|565.0| 869.2500000000007|
+-------+------------------+-------+-----+------------------+



In [39]:
result = session.execute("SELECT * FROM simulacao WHERE tempo_da_simulacao = 143 ALLOW FILTERING;")
r = list(result)
df2 = ss.createDataFrame(r)
df2.show()

+-------+------------------+-------+-----+------------------+
|rodovia|tempo_da_simulacao|  placa|pos_x|             pos_y|
+-------+------------------+-------+-----+------------------+
| BR-116|               143|ARG5B67|655.0|162.99999999999872|
| BR-116|               143|CHI6S78|745.0|             776.0|
| BR-116|               143|ECU6I78|565.0|325.35000000000014|
| BR-116|               143|GUY1X23|385.0| 435.1000000000003|
| BR-116|               143|PAR7P89|475.0| 268.8000000000002|
| BR-116|               143|SUR8W90|745.0|            749.15|
| BR-116|               143|URU0A12|835.0|             614.5|
| BR-116|               143|URU4O56|565.0| 904.5000000000007|
| BR-116|               143|URU6M78|655.0|            826.75|
+-------+------------------+-------+-----+------------------+



In [40]:
result = session.execute("SELECT * FROM simulacao WHERE tempo_da_simulacao = 144 ALLOW FILTERING;")
r = list(result)
df25 = ss.createDataFrame(r)
df25.show()

+-------+------------------+-------+-----+------------------+
|rodovia|tempo_da_simulacao|  placa|pos_x|             pos_y|
+-------+------------------+-------+-----+------------------+
| BR-116|               144|ARG5B67|655.0|146.99999999999872|
| BR-116|               144|CHI6S78|745.0|             776.0|
| BR-116|               144|ECU6I78|565.0|245.55000000000007|
| BR-116|               144|GUY1X23|385.0|459.75000000000034|
| BR-116|               144|PAR7P89|475.0| 317.5500000000002|
| BR-116|               144|SUR8W90|745.0|             753.5|
| BR-116|               144|URU0A12|835.0|             658.5|
| BR-116|               144|URU4O56|565.0| 800.5500000000006|
| BR-116|               144|URU6M78|745.0|            800.75|
+-------+------------------+-------+-----+------------------+



In [41]:
df_empty = ss.createDataFrame([], "placa: string, posicao: int, linha: int, rodovia: string, tempo_da_simulacao: int, velocidade: double, aceleracao: double, posicao_prevista: double, acima_vel: boolean, aplicaMulta: boolean")

In [42]:
import json
params = json.load(open('./mock/parametros.json'))

In [43]:
p = [[key]+list(params[key].values()) for key in params.keys()]

In [44]:
p = ss.createDataFrame(p, ["Rodovia"]+list(params[list(params.keys())[0]].keys()))

p.toPandas()

,Rodovia,sentido1Name,sentido2Name,sentido1Faixas,sentido2Faixas,probabilidadeDeEntradaDeVeiculo,velocidadeMaxima,velocidadeMinima,AceleracaoMaxima,AceleracaoMinima,probabilidadeDeTrocaDeFaixa,probabilidadeColisao,tempoColisao,kilometroInicial,interfaceGrafica,tamanhoCarro
0,BR-040,Brasilia,Rio de Janeiro,3,3,0.9,10,2,0.10,0.01,0.3,0.1,3000,100,True,74
1,BR-116,Fortaleza,Rio Grande do Sul,3,3,0.8,9,2,0.15,0.05,0.5,0.1,3000,100,False,74
2,BR-135,Belo Horizonte,São Luís,3,3,0.9,12,3,0.20,0.05,0.3,0.1,3000,100,False,74
3,BR-393,Cachoeiro de Itapemirim,Barra Mansa,2,2,0.9,8,2,0.10,0.01,0.5,0.1,3000,100,False,74


In [45]:
import queue


In [82]:
fps = 30

def processa_carro(DadosNovos, DadosCarros, colision_tolerance, colision_tolerance_quad, Parametros):
    DadosCarros = DadosCarros.drop('aplicaMulta')

    # Renomeia coluna do dado novo
    DadosNovos = DadosNovos.select(F.col('pos_y').alias('posicao_nova'), F.col('rodovia').alias('rodovia_nova'),
                                   F.col('pos_x').alias('linha_nova'),
                      F.col('tempo_da_simulacao').alias('tempo_da_simulacao_novo'),F.col('placa'))
    data_joined = DadosCarros.join(DadosNovos, on="placa", how='outer')
    data_joined = data_joined.withColumn("rodovia", coalesce(col("rodovia_nova"), col("rodovia")))
    data_joined = data_joined.withColumn("linha", coalesce(col("linha_nova"), col("linha")))
    data_joined = data_joined.drop("rodovia_nova", "linha_nova")
    
    data_joined = data_joined.withColumn("diferenca_de_posicao", col("posicao_nova") - col("posicao"))
    
    data_joined = data_joined.withColumn("diferenca_de_horario", (col("tempo_da_simulacao_novo") - col("tempo_da_simulacao"))*fps)

    data_joined = data_joined.withColumnRenamed("velocidade", "velocidade_antiga")

    data_joined = data_joined.withColumn("velocidade", col("diferenca_de_posicao") / col("diferenca_de_horario"))

    data_joined = data_joined.withColumn("diferenca_de_velocidade", col("velocidade") - col("velocidade_antiga"))

    data_joined = data_joined.withColumn("aceleracao", col("diferenca_de_velocidade") / col("diferenca_de_horario"))

    data_joined = data_joined.drop("velocidade_antiga", "posicao", "tempo_da_simulacao",
                     "diferenca_de_posicao", "diferenca_de_horario",
                     "diferenca_de_velocidade")

    data_joined = data_joined.withColumnRenamed("posicao_nova", "posicao")

    data_joined = data_joined.withColumnRenamed("tempo_da_simulacao_novo", "tempo_da_simulacao")

    data_joined = data_joined.withColumn("posicao_prevista", col("posicao")\
                           + col("velocidade")*colision_tolerance\
                           + col("aceleracao")*colision_tolerance_quad)

    Velocidades_Maximas = Parametros.select(F.col('rodovia'), F.col("VelocidadeMaxima"))
    data_joined = data_joined.join(Velocidades_Maximas, on="rodovia", how="left")

    acima_vel_df = data_joined.select(F.col('placa'), F.col('acima_vel').alias('acima_vel_antigo'))
    data_joined = data_joined.withColumn("acima_vel", F.when(F.abs(data_joined["velocidade"]) > F.abs(fps/data_joined["VelocidadeMaxima"]), 1).otherwise(0))

    acima_vel_df = acima_vel_df.join(data_joined.select(F.col('placa'), F.col('acima_vel').alias('acima_vel_novo')), on='placa', how="left")
    acima_vel_df = acima_vel_df.withColumn("aplicaMulta", (F.col('acima_vel_antigo')==0) &  (F.col('acima_vel_novo')==1) )

    data_joined = data_joined.join(acima_vel_df.select(F.col('placa'), F.col('aplicaMulta')), on='placa', how='left')

    data_joined = data_joined.drop("VelocidadeMaxima")
    return data_joined
    

In [83]:
df_empty.show()

+-----+-------+-----+-------+------------------+----------+----------+----------------+---------+-----------+
|placa|posicao|linha|rodovia|tempo_da_simulacao|velocidade|aceleracao|posicao_prevista|acima_vel|aplicaMulta|
+-----+-------+-----+-------+------------------+----------+----------+----------------+---------+-----------+
+-----+-------+-----+-------+------------------+----------+----------+----------------+---------+-----------+



In [84]:
df3 = processa_carro(df, df_empty, 1, 0.5, p)

In [85]:
df3 = processa_carro(df2, df3, 1, 0.5, p)

In [86]:
df4 = processa_carro(df25, df3, 1, 0.5, p).toPandas()

In [87]:
df4

,placa,rodovia,linha,aceleracao,posicao_prevista,acima_vel,posicao,tempo_da_simulacao,velocidade,aplicaMulta
0,ARG5B67,BR-116,655.0,-0.011111,146.461111,0,147.00,144,-0.533333,False
1,CHI6S78,BR-116,745.0,0.048889,776.024444,0,776.00,144,0.000000,False
2,ECU6I78,BR-116,565.0,-0.349333,242.715333,0,245.55,144,-2.660000,False
3,GUY1X23,BR-116,385.0,-0.111333,460.516000,0,459.75,144,0.821667,False
4,PAR7P89,BR-116,475.0,-0.075333,319.137333,0,317.55,144,1.625000,False
5,SUR8W90,BR-116,745.0,0.004833,753.647417,0,753.50,144,0.145000,False
6,URU0A12,BR-116,835.0,0.088889,660.011111,0,658.50,144,1.466667,False
7,URU4O56,BR-116,565.0,-0.154667,797.007667,1,800.55,144,-3.465000,True
8,URU6M78,BR-116,745.0,NaN,NaN,0,800.75,144,-0.866667,False
